# Clustering Crypto

In [32]:
conda activate pyvizenv


Note: you may need to restart the kernel to use updated packages.


In [33]:
pip install hvplot

In [34]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [35]:
# Initial imports
import requests
import pandas as pd
import numpy as np
import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [36]:
# Use the following endpoint to fetch json data

# url = "https://min-api.cryptocompare.com/data/all/coinlist"
# response = requests.get(url).json()

In [37]:
# Create a DataFrame

# df = pd.DataFrame(response['Data']).T
# df.head()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,AssetLaunchDate,MaxSupply,MktCapPenalty,IsUsedInDefi,IsUsedInNft,PlatformType,BuiltOn,SmartContractAddress,DecimalPoints,Difficulty
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,0000-00-00,0,0,0,0,NaN,NaN,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,2017-07-01,300,0,0,0,token,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18,NaN
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,0000-00-00,0,0,0,0,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,0000-00-00,0,0,0,0,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
# Alternatively, use the provided csv file:
file_path = Path("crypto_data.csv")

# Create a DataFrame
df = pd.read_csv(file_path, index_col=0)
df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


### Data Preprocessing

In [72]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'

df = df[['CoinName', 'Algorithm', 'IsTrading', 'ProofType', 'TotalCoinsMined', 'TotalCoinSupply']]

df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [73]:
# Keep only cryptocurrencies that are trading

df = df[df["IsTrading"] == True]
print(df.shape)

df.head()

(1144, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [74]:
# Keep only cryptocurrencies with a working algorithm

df = df[df["Algorithm"] != "N/A"]
print(df.shape)

df.head()

(1144, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [75]:
# Remove the "IsTrading" column

df.drop("IsTrading", axis=1, inplace=True)
print(df.shape)

df.head()

(1144, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [76]:
# Remove rows with at least 1 null value

df = df.dropna(axis=0, how="any")
print(df.shape)

df.head()

(685, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [77]:
# Remove rows with cryptocurrencies having no coins mined

df = df[df["TotalCoinsMined"] > 0]
print(df.shape)

df.head()

(532, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [78]:
# Drop rows where there are 'N/A' text values

df = df[df.iloc[:] != 'N/A'].dropna()
df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [79]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df

Coin_name = pd.DataFrame(df["CoinName"], index=df.index)
print(Coin_name.shape)

Coin_name.head()

(532, 1)


,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [80]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm

df = df.drop("CoinName", axis=1)
print(df.shape)

df.head()

(532, 4)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [81]:
# Create dummy variables for text features

Dummy_v = pd.get_dummies(data=df, columns=["Algorithm", "ProofType"])
print(Dummy_v.shape)

Dummy_v.head()

(532, 98)


,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
# Standardize data

Standard_d = StandardScaler().fit_transform(Dummy_v)

Dummy_v[:5]

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Reducing Dimensions Using PCA

In [83]:
# Use PCA to reduce dimension to 3 principal components

n_comp = 3
pca = PCA(n_components=n_comp)
principal_c = pca.fit_transform(Standard_d)

principal_c

array([[-0.33516856,  1.08522512, -0.56782594],
       [-0.31851568,  1.0854978 , -0.5682033 ],
       [ 2.28231917,  1.67770553, -0.59557689],
       ...,
       [ 0.3131339 , -2.31279463,  0.39061023],
       [-0.16626965, -2.06298341,  0.49360254],
       [-0.25197442,  0.70154047, -0.26801679]])

In [84]:
# Create a DataFrame with the principal components data

col_names = [f"PC {i}" for i in range(1, n_comp + 1)]
pc_df = pd.DataFrame(principal_c, columns=col_names, index=df.index)
print(pc_df.shape)

pc_df.head(10)

(532, 3)


,PC 1,PC 2,PC 3
42,-0.335169,1.085225,-0.567826
404,-0.318516,1.085498,-0.568203
1337,2.282319,1.677706,-0.595577
BTC,-0.156315,-1.306916,0.191356
ETH,-0.155185,-2.072452,0.342834
LTC,-0.160656,-1.107064,-0.010215
DASH,-0.418160,1.342744,-0.524861
XMR,-0.161933,-2.222318,0.480934
ETC,-0.153629,-2.072540,0.342816
ZEC,-0.166269,-2.062983,0.493603


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [85]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pc_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot

elbow_curve = {"k": k, "inertia": inertia}
elbow_df = pd.DataFrame(elbow_curve)
elbow_df.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

C:\Users\alish\anaconda3\ANACONDA\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [86]:
# Initialize the K-Means model

model = KMeans(n_clusters=4, random_state=0)

# Fit the model

model.fit(pc_df)

# Predict clusters

predict_c = model.predict(pc_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features

cluster_df = pd.concat([df, pc_df], axis=1, sort=False)
cluster_df["CoinName"] = Coin_name["CoinName"]
cluster_df["Class"] = model.labels_
print(cluster_df.shape)

cluster_df.head()

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.335169,1.085225,-0.567826,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.318516,1.085498,-0.568203,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.282319,1.677706,-0.595577,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.156315,-1.306916,0.191356,Bitcoin,3
ETH,Ethash,PoW,1.076842e+08,0,-0.155185,-2.072452,0.342834,Ethereum,3


### Visualizing Results

#### 3D-Scatter with Clusters

In [89]:
# Create a 3D-Scatter with the PCA data and the clusters

scatter_d = px.scatter_3d(
    cluster_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    hover_name="CoinName",
    hover_data=["Algorithm"],
    width=800,
)
scatter_d.update_layout(legend=dict(x=0, y=1))
scatter_d.show()

#### Table of Tradable Cryptocurrencies

In [91]:
# Table with tradable cryptos

cluster_df[
    [
        "CoinName",
        "Algorithm",
        "ProofType",
        "TotalCoinSupply",
        "TotalCoinsMined",
        "Class",
    ]
].hvplot.table()

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [92]:
# Print the total number of tradable cryptocurrencies

print(f"There are a total of {cluster_df.shape[0]} tradable cryptocurrencies.")

There are a total of 532 tradable cryptocurrencies.


#### Scatter Plot with Tradable Cryptocurrencies

In [93]:
# Scale data to create the scatter plot

mm_scale = MinMaxScaler()
scatter_data = mm_scale.fit_transform(
    cluster_df[["TotalCoinSupply","TotalCoinsMined"]]
)
scatter_df = pd.DataFrame(
    scatter_data, columns=["TotalCoinSupply","TotalCoinsMined"], index=cluster_df.index
)
scatter_df["CoinName"] = cluster_df["CoinName"]
scatter_df["Class"] = cluster_df["Class"]

scatter_df.head()

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,3
ETH,0.000000e+00,0.000109,Ethereum,3


In [94]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
scatter_df.hvplot.scatter(
    x="TotalCoinsMined", y="TotalCoinSupply", hover_cols=["CoinName"], by="Class"
)

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)